In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import gym
import random
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
from PPO import PPO, PPOContinuous
import pdb

env = gym.make("Pendulum-v0")

env.seed(10)

state_size = env.observation_space.shape[0]
action_size =env.action_space.shape[0]

print(state_size)
print(action_size)

3
1


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [ ]:
# PPO Settings
update_every = 4000
num_learn = 80
win_condition = 0

# Agent settings
hidden_size=64
epsilon=0.2
entropy_beta=0.01
gamma=0.99
lr=0.0003

agent = PPOContinuous(state_size, action_size, hidden_size=hidden_size, epsilon=epsilon, entropy_beta=entropy_beta, gamma=gamma, lr=lr)

def train(n_episodes=2000, max_t=1500):
  steps = 0
  scores_deque = deque(maxlen=100)
  scores = []
  average_scores = []
  max_score = -np.Inf

#   agent = PPO(state_size, action_size, hidden_size=hidden_size, epsilon=epsilon, entropy_beta=entropy_beta, gamma=gamma, lr=lr)

  for episode in range(1, n_episodes+1):
    state = env.reset()
    score = 0
    
    for t in range(max_t):
      steps += 1

      actions_tensor, log_prob = agent.act(torch.FloatTensor(state))
      actions = actions_tensor.cpu().data.numpy().flatten()
      next_state, reward, done, _ = env.step(actions)
      
      agent.mem.add(torch.FloatTensor(state), actions_tensor, reward, log_prob, done)

      # Update 
      state = next_state
      score += reward

      if steps >= update_every:
        agent.learn(num_learn)
        agent.mem.clear()
        steps = 0

      if done:
        break
    
    # Book Keeping
    scores_deque.append(score)
    scores.append(score)
    average_scores.append(np.mean(scores_deque))
      
    print("\rEpisode {}	Average Score: {:.2f}	Score: {:.2f}".format(episode, np.mean(scores_deque), score), end="")

    if episode % 100 == 0:
      print("\rEpisode {}	Average Score: {:.2f}".format(episode, np.mean(scores_deque)))   
    
    if np.mean(scores_deque) > win_condition:
      print("\rEnvironment Solved in {} episodes!".format(episode))
      break


  return scores, average_scores
    
scores, average_scores = train()

Episode 19	Average Score: -1678.77	Score: -1834.50

/Users/darylrodrigo/Desktop/rl_lib/Policy Gradient/src/PPO/PPO.py:129: UserWarning: Using a target size (torch.Size([4000])) that is different to the input size (torch.Size([4000, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = -torch.min(surrogate_1, surrogate_2) + F.mse_loss(values, discounted_returns) - self.entropy_beta*entropy


Episode 100	Average Score: -1649.09	Score: -1633.10
Episode 119	Average Score: -1654.70	Score: -1632.33

In [ ]:
plt.plot(scores)
plt.plot(average_scores)

In [ ]:
for episode in range(3):
    state = env.reset()
    score = 0
    input()

    for t in range(700):
        actions_tensor, log_prob = agent.act(torch.FloatTensor(state))
        actions = actions_tensor.cpu().data.numpy().flatten()
        next_state, reward, done, _ = env.step(actions_tensor)

        
        score += reward
        
        env.render()
        
        if done:
            print(score)
            break;
        
        state = next_state

env.close()